In [1]:
from src import SQL_Executor, SQL_Queries
import pandas as pd

executor = SQL_Executor()
query = SQL_Queries.Payment()

In [2]:
df = executor.execute_query(query.define_dpd_aggregate())
df

,transaction_id,installment_number,dpd
0,1,1,5
1,1,2,2
2,1,3,2
3,1,4,0
4,2,1,3
...,...,...,...
214823,44623,8,-2
214824,44623,9,-32
214825,44623,10,-62
214826,44623,11,-92


In [3]:
dpd90_df = df.groupby('transaction_id').agg({
    'dpd': 'max'
}).reset_index()

dpd90_df['dpd90'] = (dpd90_df['dpd'] >= 90).astype(int)
dpd90_df

,transaction_id,dpd,dpd90
0,1,5,0
1,2,5,0
2,3,5,0
3,4,1261,1
4,5,1291,1
...,...,...,...
44618,44619,217,1
44619,44620,216,1
44620,44621,215,1
44621,44622,210,1


In [4]:
t_df = executor.execute_query(query.calculate_dpd90_trend())
result_df = t_df.merge(dpd90_df, on='transaction_id', how='left')

In [5]:
labels = ['18-25', '26-35', '36-50', '51+']
bins = [18, 26, 36, 51, 100]
group_age = pd.cut(result_df['age'], bins=bins, labels=labels)
dpd90_rates_age = result_df.groupby(group_age, observed=True)['dpd90'].agg(
    dpd90_rate_age = lambda x: (x.sum() * 100).round(2)
).reset_index()

dpd90_rates_age

,age,dpd90_rate_age
0,18-25,959400
1,26-35,438600
2,36-50,199300
3,51+,141000


In [6]:
bins = [0, 15000, 30000, 60000, float('inf')]
labels = ['0-15k', '15k-30k', '30k-60k', '60k+']

group_income = pd.cut(result_df['income'], bins=bins, labels=labels)
dpd90_rates_income = result_df.groupby(group_income, observed=True)['dpd90'].agg(
    dpd90_rate_income = lambda x: (x.mean() * 100).round(2)
).reset_index()

dpd90_rates_income

,income,dpd90_rate_income
0,0-15k,55.61
1,15k-30k,56.06
2,30k-60k,34.75
3,60k+,30.75
